In [1]:
import requests
from datetime import datetime, timedelta

In [2]:
CLOUDFLARE_API_KEY ="5XdlglW_vpz3X9PoPozByHhJ3PK2aSJgUCBbtKJ8"
CF_MODEL = "@cf/mistral/mistral-7b-instruct-v0.1"
CF_API_URL = "https://api.cloudflare.com/client/v4/accounts/{account_id}/ai/run/{model_name}"
MAX_TOKENS = 1800
mapbox_token = "pk.eyJ1IjoiYXJ5YW5zaGFybWE4MTAiLCJhIjoiY203eXRtY29yMGRvYTJpc2FoMDI5bjAxMyJ9.edEiAbWX4ZcRs6qq921oZQ"
tomtom_api_key = 'k8wn0ShnUx6ZmuDGF9x4KCWR6bAvWmPQ'
DEFAULT_DAYS = 3

In [ ]:
def get_coordinates(location, mapbox_token):
    url = f"https://api.mapbox.com/geocoding/v5/mapbox.places/{location}.json"
    params = {"access_token": mapbox_token, "limit": 1}
    try:
        response = requests.get(url, params=params, timeout=10)
        if response.status_code == 200:
            data = response.json()
            features = data.get("features", [])
            if features:
                center = features[0].get("center")
                if isinstance(center, list) and len(center) >= 2:
                    return [float(center[0]), float(center[1])]
    except Exception as e:
        print(f"Geocoding error: {str(e)}")
    return None

In [4]:
def get_mapbox_pois(location, category, limit, token):
    coords = get_coordinates(location, token)
    if not coords or len(coords) < 2:
        return []

    try:
        url = f"https://api.mapbox.com/geocoding/v5/mapbox.places/{category}.json"
        params = {
            "access_token": token,
            "limit": limit,
            "proximity": f"{coords[0]},{coords[1]}"
        }
        response = requests.get(url, params=params, timeout=10)
        if response.status_code == 200:
            return [
                f["text"] for f in response.json().get("features", [])
                if f.get("text")
            ]
    except Exception as e:
        print(f"Mapbox POI error: {str(e)}")
    return []


def get_tomtom_pois(coords, category, limit, api_key):
    if not coords or len(coords) < 2:
        return []

    try:
        url = "https://api.tomtom.com/search/2/categorySearch/.json"
        params = {
            "key": api_key,
            "lat": coords[1],
            "lon": coords[0],
            "limit": limit,
            "categorySet": "7376"
        }
        response = requests.get(url, params=params, timeout=10)
        if response.status_code == 200:
            return [
                r["poi"]["name"] for r in response.json().get("results", [])
                if r.get("poi", {}).get("name")
            ]
    except Exception as e:
        print(f"TomTom POI error: {str(e)}")
    return []

In [5]:
def get_weather_context(coords):
    try:
        url = "https://api.open-meteo.com/v1/forecast"
        params = {
            "latitude": coords[1],
            "longitude": coords[0],
            "current_weather": "true"
        }
        response = requests.get(url, params=params, timeout=10)
        if response.status_code == 200:
            weather = response.json().get("current_weather", {})
            return f"{weather.get('temperature', 'N/A')}°C, Wind: {weather.get('windspeed', 'N/A')}km/h"
    except Exception:
        pass
    return "Weather info unavailable"

In [6]:
def get_location_info(location_name):
    try:
        url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{location_name.replace(' ', '_')}"
        response = requests.get(url, timeout=5)
        if response.status_code == 200:
            data = response.json()
            return data.get('extract', '')
    except Exception:
        pass
    return f"{location_name} is a fascinating destination with rich history and cultural significance."

In [7]:
def collect_travel_data(location, num_days, mapbox_token, tomtom_key):

    coords = get_coordinates(location, mapbox_token)
    if not coords:
        raise ValueError("Could not retrieve coordinates for location")

    return {
        "location_info": get_location_info(location.split(',')[0].strip()),
        "hotels": list(set(
            get_mapbox_pois(location, "hotel", 10, mapbox_token) +
            get_tomtom_pois(coords, "hotel", 10, tomtom_key)
        ))[:10],
        "attractions": list(set(
            get_mapbox_pois(location, "attraction", num_days*4, mapbox_token) +
            get_tomtom_pois(coords, "attraction", num_days*4, tomtom_key)
        ))[:num_days*4],
        "transportation": get_transportation_info(coords, mapbox_token),
        "weather": get_weather_context(coords)
    }

In [8]:
def get_transportation_info(coords, mapbox_token):
    try:
        url = "https://api.mapbox.com/directions-matrix/v1/mapbox/driving"
        params = {
            "access_token": mapbox_token,
            "sources": f"{coords[0]},{coords[1]}",
            "annotations": "duration"
        }
        response = requests.get(url, params=params, timeout=10)
        if response.status_code == 200:
            durations = response.json().get("durations", [[]])[0]
            return f"Average travel time: {sum(durations)/len(durations)/60:.1f} mins" if durations else ""
    except Exception:
        pass
    return "Typical travel time: 15-45 mins"

In [9]:
def format_prompt(rag_data, user_query, num_days):
    return f"""<s>[INST] Create a detailed {num_days}-day travel itinerary with these components:

Destination Context: {rag_data['location_info']}

Key Information:
- Available Hotels: {', '.join(rag_data['hotels'][:3])}
- Top Attractions: {', '.join(rag_data['attractions'][:6])}
- Transportation Details: {rag_data['transportation']}
- Current Weather: {rag_data['weather']}
- User Preferences: {user_query}

Structure Requirements:
1. Daily schedule with time blocks (Morning/Afternoon/Evening)
2. Historical context for each main attraction
3. Cultural significance explanations
4. Local dining suggestions near activities
5. Hotel recommendations with proximity notes

Include practical tips and safety considerations where relevant.
[/INST]</s>
"""

In [12]:
def generate_itinerary(prompt, account_id):
    headers = {
        "Authorization": f"Bearer {CLOUDFLARE_API_KEY}",
        "Content-Type": "application/json"
    }

    payload = {
        "prompt": prompt,
        "max_tokens": MAX_TOKENS,
        "temperature": 0.7,
        "top_p": 0.9,
        "stream": False
    }

    try:
        response = requests.post(
            CF_API_URL.format(account_id=account_id, model_name=CF_MODEL),
            headers=headers,
            json=payload,
            timeout=300
        )
        response.raise_for_status()
        return response.json().get("result", {}).get("response", "Error generating itinerary")
    except Exception as e:
        return f"API Error: {str(e)}"

In [ ]:
# User interaction flow remains the same until generation
location = input("\nEnter destination (e.g., 'Rome, Italy'): ").strip()
user_query = input("Describe your travel preferences: ").strip()

try:
    num_days = int(input("Number of days (3-10): ").strip())
    num_days = max(min(num_days, 10), 1)
except ValueError:
    num_days = DEFAULT_DAYS
    print(f"Using default duration: {DEFAULT_DAYS} days")

print("Collecting travel data...")
try:

    CLOUDFLARE_ACCOUNT_ID = "ab32823f1a2c6422bfa2771f82f979bc"

    rag_data = collect_travel_data(location, num_days, mapbox_token, tomtom_api_key)
    prompt = format_prompt(rag_data, user_query, num_days)

    print("Generating itinerary via Cloudflare AI...")
    itinerary = generate_itinerary(prompt, CLOUDFLARE_ACCOUNT_ID)

    print("\n=== TRAVEL PLAN ===\n")
    print(itinerary)

except Exception as e:
    print(f"Error: {str(e)}")


Enter destination (e.g., 'Rome, Italy'): Kolkata
Describe your travel preferences: plan my trip
Number of days (3-10): 7
Generating itinerary via Cloudflare AI...

=== TRAVEL PLAN ===

 Day 1: Arrival in Kolkata

Morning:
- Arrive at Netaji Subhas Chandra Bose International Airport. The airport is located about 20 km north of the city center.
- Check-in at Hôtel de Ville-Quinconces, which is centrally located and convenient for exploring the city. (Travel time: 30 mins)

Afternoon:
- After settling in, take a leisurely walk around the nearby Maidan, the largest city park in Kolkata, and visit the Victoria Memorial. (15-minute walk)
- Have lunch at Bhojohori Manna, a popular Bengali restaurant located near the Maidan.

Evening:
- Visit Vijay Smarak, a memorial dedicated to the soldiers of the Indian Armed Forces who have made the supreme sacrifice in various wars and peacekeeping missions. (10-minute taxi ride)

Day 2: Exploring Historical Kolkata

Morning:
- Start your day with a visi